# Food security portal datasets

Running this notebooks collects all data sources from http://www.foodsecurityportal.org/ maintained by the EU. Available datasets are automatically scrapped from homepage http://www.foodsecurityportal.org/api/about.

## Settings

In [11]:
# helpers
from pathlib import Path
from urllib.request import Request, urlopen
import requests
from datetime import datetime
import shutil
import tqdm

# data processing
import pandas as pd
from bs4 import BeautifulSoup

# conf
import sys
from path_manipulation import get_to_root

sys.path[0] = get_to_root(3,sys.path[0])
from config.config import config
sources = config.sources

In [22]:
def scarp_all_datasets_from_foodSecurityPortal(
    url=sources.food_security_api_url,
):
    # Return a Python list that contains all links pointing to datasets to date
    headers = {"User-Agent": "Mozilla/5.0"}
    req = Request(url=url, headers=headers)
    with urllib.request.urlopen(req) as page:
        html = BeautifulSoup(page.read().decode("utf-8"), "html.parser")
        all_links_in_page = html.find_all("a")
        all_not_empty_links = [
            link.get("href")
            for link in all_links_in_page
            if (not link.get("href") is None)
        ]
        dataset_links = [
            link
            for link in all_not_empty_links
            if (
                (link != "/api")
                and (not "documentation" in link)
                and (not "about" in link)
                and ("api" in link)
            )
        ]
        return dataset_links


def get_csv_file_from_dataset(
    dataset_links,
    url=sources.food_security_url,
    output_path=sources.food_security_output_folder_path,
):
    # Simply download corresponding .csv file while removing existing ones
    # Could not think of better solution since it's hard to check for updates...

    for dataset_url in dataset_links:
        filename = dataset_url.split("/")[-1] + ".csv"
        print("---", dataset_url, filename)
        r = requests.get(url + dataset_url + ".csv")
        with open(output_path / filename, "wb") as outfile:
            outfile.write(r.content)

## Get all datasets from http://www.foodsecurityportal.org/

In [23]:
# Get the datasets urls
dataset_links = scarp_all_datasets_from_foodSecurityPortal()
# Download the csv, save it
get_csv_file_from_dataset(dataset_links)

--- http://docs.ckan.org/en/2.8/api/ .csv


FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\charl\\Repositories\\batch8_actioncontrelafaim\\src\\notebooks\\collection\\..\\..\\..\\data\\clean\\food_security\\.csv'